# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [8]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd 
import json

In [9]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [10]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [11]:
# assign the uk_food database to a variable name
db = mongo.uk_food

In [12]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [13]:
# assign the collection to a variable
est = db.establishments

In [14]:
# review a document in the establishments collection
est.find_one()

{'_id': ObjectId('646c4b4d213ff3b048ba09d2'),
 'FHRSID': 289353,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6QH',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2016-09-13T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.194762, 'latitude': 51.085797},
 'RightToReply': '',
 'Distance': 4591.871474174837,
 'NewRatingPending': False,
 'meta': {'dataSo

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [15]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [16]:
# Insert the new restaurant into the collection
est.insert_one(new_restaurant)

In [17]:
# Check that the new restaurant was inserted
check = list(est.find({"BusinessName": "Penang Flavours"}))
pprint(check)

[{'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  'Distance': 4623.972328074718,
  'LocalAuthorityCode': '511',
  'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
  'LocalAuthorityName': 'Greenwich',
  'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
  'NewRatingPending': True,
  'Phone': '',
  'PostCode': 'SE18 7DY',
  'RatingValue': None,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('646c4d94f977a5695c56beb2'),
  'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
  'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}},
 {'AddressLine1': 'Penang Flavours',
  'AddressLine2': '146A Plumstead Rd',
  'AddressLine3': 'London',
  'AddressLine4': '',
  'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'Busines

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [18]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
#Define the query
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
params = {
    "BusinessTypeID": 1,
    "BusinessType": 1
}

#Save the results
results = est.find_one(query, params)

#Find the Business Type ID
print(f"The Business Type ID for {results['BusinessType']} Business Type is: {results['BusinessTypeID']}")

The Business Type ID for Restaurant/Cafe/Canteen Business Type is: 1


3. Update the new restaurant with the `BusinessTypeID` you found.

In [19]:
# Update the new restaurant with the correct BusinessTypeID
bt_id = results["BusinessTypeID"]

est.update_one({"BusinessName": "Penang Flavours"}, {'$set': {'BusinessTypeID': bt_id}})

In [20]:
# Confirm that the new restaurant was updated
check2 = list(est.find({"BusinessName": "Penang Flavours"}))
print(f"The Business Type ID for {check2[0]['BusinessName']} is {check2[0]['BusinessTypeID']}")

The Business Type ID for Penang Flavours is 1


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [21]:
# Find how many documents have LocalAuthorityName as "Dover"
query_dover = {"LocalAuthorityName": "Dover"}

print(f"There are {est.count_documents(query_dover)} documents from the Denver Local Authority.")

There are 0 documents from the Denver Local Authority.


In [22]:
# Delete all documents where LocalAuthorityName is "Dover"
est.delete_many(query_dover)

In [23]:
# Check if any remaining documents include Dover
print(f"There are {est.count_documents(query_dover)} documents from the Denver Local Authority.")

There are 0 documents from the Denver Local Authority.


In [24]:
# Check that other documents remain with 'find_one'
est.find_one()

{'_id': ObjectId('646c4b4d213ff3b048ba09d2'),
 'FHRSID': 289353,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000002468',
 'BusinessName': 'St Marys COE (aided) Primary School',
 'BusinessType': 'School/college/university',
 'BusinessTypeID': 7845,
 'AddressLine1': 'St Marys Church Of England Primary School',
 'AddressLine2': 'Warren Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6QH',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2016-09-13T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.194762, 'latitude': 51.085797},
 'RightToReply': '',
 'Distance': 4591.871474174837,
 'NewRatingPending': False,
 'meta': {'dataSo

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [25]:
# Change the data type from String to Decimal for longitude
est.update_many({}, [{'$set': {"geocode.longitude": {'$toDouble': "$geocode.longitude"}}}])

In [27]:
# Change the data type from String to Decimal for latitude
est.update_many({}, [{'$set': {"geocode.latitude": {'$toDouble': "$geocode.latitude"}}}])

In [36]:
# Change the data type from Integer to String for RatingValue
est.update_many({}, [{'$set': {"RatingValue": {"$convert": {'input': "$RatingValue", 'to': "int", 'onError': "An Error occurred", 'onNull': "null"}}}}])

In [48]:
# Check that the coordinates are now numbers
query_num = {}
params_num = {
    "geocode.longitude": 1,
    "geocode.latitude": 1,
    "RatingValue": 1
}

#Save the results
results_num = list(est.find(query_num, params_num))

#Pretty print a result
pprint(results_num[0])

{'RatingValue': 5,
 '_id': ObjectId('646c4b4d213ff3b048ba09d2'),
 'geocode': {'latitude': 51.085797, 'longitude': 1.194762}}


In [49]:
#Print the data types
print(f"The data type of the longitude field is: {type(results_num[0]['geocode']['longitude'])}")
print(f"The data type of the latitude field is: {type(results_num[0]['geocode']['latitude'])}")
print(f"The data type of the RatingValue field is: {type(results_num[0]['RatingValue'])}")

The data type of the longitude field is: <class 'float'>
The data type of the latitude field is: <class 'float'>
The data type of the RatingValue field is: <class 'int'>


In [45]:

for j in range(len(results_num)):
    if isinstance(results_num[j]["RatingValue"], str) == True:
        print(j)

38
43
79
100
109
127
143
145
152
163
175
176
198
234
250
251
272
313
317
335
396
557
573
586
594
647
703
710
715
733
793
819
820
826
844
849
859
862
878
976
978
1019
1041
1058
1104
1137
1165
1222
1230
1235
1241
1247
1248
1268
1291
1436
1523
1625
1722
1741
1754
1769
1777
1778
1789
1820
1856
1864
1867
1876
1910
1917
1944
1950
1970
1973
1980
1996
2000
2078
2169
2176
2189
2195
2226
2291
2296
2302
2332
2362
2410
2412
2427
2475
2545
2582
2638
2655
2685
2687
2688
2708
2712
2741
2742
2744
2776
2777
2780
2793
2803
2805
2871
2879
2882
2890
2896
2898
3027
3028
3074
3088
3099
3107
3114
3120
3158
3168
3204
3228
3230
3274
3289
3299
3310
3343
3358
3368
3383
3404
3425
3451
3462
3480
3489
3523
3540
3541
3576
3577
3585
3596
3611
3630
3650
3696
3699
3713
3720
3723
3737
3749
3756
3802
3815
3838
3857
3877
3878
3881
3882
3904
3929
3945
3951
3955
3966
3977
3986
3988
4006
4015
4024
4027
4032
4035
4045
4057
4115
4124
4142
4143
4146
4173
4174
4190
4191
4217
4232
4247
4294
4306
4309
4352
4356
4376
4386
4446
4466

In [46]:
results_num[38]

{'_id': ObjectId('646c4b4d213ff3b048ba09fb'),
 'RatingValue': 'An Error occurred',
 'geocode': {'longitude': 1.18590330311705, 'latitude': 51.0783519967076}}

In [47]:
print(f"The data type of the longitude field is: {type(results_num[j]['geocode']['longitude'])}")
print(f"The data type of the latitude field is: {type(results_num[j]['geocode']['latitude'])}")
print(f"The data type of the RatingValue field is: {type(results_num[j]['RatingValue'])}")

The data type of the longitude field is: <class 'float'>
The data type of the latitude field is: <class 'float'>
The data type of the RatingValue field is: <class 'str'>
